In [3]:
import pandas as pd 
#loading data and removing cloumn 0 because it does not contain any index
data = pd.read_csv("Assignment_data/Data_Q3/D48.csv",index_col=0)

In [4]:
import numpy as np 
from sklearn.model_selection import train_test_split

In [5]:
#separating test set and train set
import hashlib
def test_set_check(identifier,test_ratio,hash):
    return bytearray(hash(np.int64(identifier)).digest())[-1]  < 51
def split_train_test_by_id(data,test_ratio,id_column,hash):
    ids=data[id_column]
    in_test_set=ids.apply(lambda id_:test_set_check(id_,test_ratio,hash))
    return in_test_set


In [6]:
data=data.reset_index()
in_test_set=split_train_test_by_id(data,0.66,"index",hashlib.md5)
test_set=data[in_test_set]
train_set=data[~in_test_set]
#deleting index columns from both test set and train set
del train_set['index']
del test_set['index']

In [7]:
#convert train set to matrix
train_set_array = train_set.as_matrix()

In [8]:
#deleting predicted values from test set and converting both copies of test set to matrix
test_set_copy = test_set
test_set_copy = test_set_copy.as_matrix()
#stored test result to compare later with predictions 
testforcheck = np.array((420,1))
testforcheck = test_set['9'].values
del test_set['9']

test_set_array = test_set.as_matrix()

In [9]:
#distance between two sets
import math
def euclidiendistance(instance1, instance2, length ):
    distance = 0
    for x in range(length):
        distance += pow(instance1[x] - instance2[x],2)
    return math.sqrt(distance)

In [10]:
#get the distance of test instance from all training instances and return the minimum distance
import operator
def getdistance(trainingSet, testInstance):
    distances = []
    length = len(testInstance)
    for x in range(len(trainingSet)):
        dist = euclidiendistance(testInstance,trainingSet[x],length)
        distances.append((trainingSet[x],dist))
    distances.sort(key=operator.itemgetter(1))
    return distances[0][0]

In [11]:
#getting training set with minimum distance from all test instances one by one 
predictions = []
for i in range(len(test_set_array)):
    neighbour = getdistance(train_set_array,test_set_array[1])
    predictions.append(neighbour[8])

In [12]:
#get accuracy by finding percentage of correct predictions 
def getAccuracy(testSet,predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct +=1
    return (correct/float(len(testSet)))*100
    

In [13]:
accuracy = getAccuracy(test_set_copy,predictions)
accuracy

81.9047619047619

In [14]:
from sklearn.metrics import accuracy_score

prediction_array = np.asarray(predictions)

prediction_array  = prediction_array.reshape((420,1))
accuracy_score(testforcheck,prediction_array)

0.81904761904761902

In [15]:
from sklearn.metrics import precision_recall_fscore_support
#function returns precision, recall,fscore
precision_recall_fscore_support(testforcheck,prediction_array, average='binary')

(0.81904761904761902, 1.0, 0.90052356020942403, None)

In [17]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
false_positive_rate, true_positive_rate, thresholds = roc_curve(testforcheck, prediction_array)
print auc(false_positive_rate, true_positive_rate)

0.5
